# Predict 40 Yard Dash Times with our Model

In a previous notebook we tested several models and saved the one that served our purposes. In this notebook we'll create the dataset and use our model to predict 40 yard dash times.

In [94]:
# Import important standard libraries
import pandas as pd
import numpy as np

# import itertools
import itertools
from datetime import datetime

In [95]:
# Read original data file
combine_raw_df  = pd.read_csv('resources/combine_df.csv').dropna(subset=['Height (in)', 'Weight (lbs)','Broad Jump (in)'])

combine_raw_df.columns

Index(['Year', 'Name', 'College', 'POS', 'Height (in)', 'Weight (lbs)', 'BMI',
       '40 Yard', 'Bench Press', 'Vert Leap (in)', 'Broad Jump (in)',
       'Shuttle', '3Cone'],
      dtype='object')

In [96]:
# Create lists of values that will be our input data.
# use the min and max values from the orignal data as the range.
height = [i for i in range(int(combine_raw_df['Height (in)'].min()*(0.95)),
                               int(combine_raw_df['Height (in)'].astype('int').max()*(1.05)),1)]
weight = [i for i in range(int(combine_raw_df['Weight (lbs)'].min()*(0.95)),
                               int(combine_raw_df['Weight (lbs)'].astype('int').max()*(1.05)),1)]
broad_jump = [i for i in range(int(combine_raw_df['Broad Jump (in)'].min()*(0.95)),
                               int(combine_raw_df['Broad Jump (in)'].astype('int').max()*(1.05)),1)]

# Combine the values into all posiblle combinations

combo_list = list(itertools.product(height,weight,broad_jump))

predictions_df = pd.DataFrame(combo_list, columns=['Height (in)', 'Weight (lbs)','Broad Jump (in)'])
predictions_df.insert(0,'Year',datetime.now().year)

# Print the dataframe
predictions_df.head()

,Year,Height (in),Weight (lbs),Broad Jump (in)
0,2024,58,136,6
1,2024,58,136,7
2,2024,58,136,8
3,2024,58,136,9
4,2024,58,136,10


In [97]:
predictions_df.shape

(1118880, 4)

In [98]:
# Import pickle
import pickle

# Import our model from the other Regression ML notebook
loaded_model = pickle.load(open('combine_score_predictive_model.pkl', 'rb'))


In [99]:
predictions = loaded_model.predict(predictions_df)

predictions

array([4.75842568, 4.75842568, 4.75842568, ..., 5.11703416, 5.11703416,
       5.11703416])

In [100]:
# Get predictions from each tree
predictions_per_tree = np.array([tree.predict(predictions_df) for tree in loaded_model.estimators_])

# Calculate prediction mean
mean_prediction = np.mean(predictions_per_tree, axis=0)

# Calculate prediction intervals (e.g., 90% confidence)
lower_bound = np.percentile(predictions_per_tree, 5, axis=0)
upper_bound = np.percentile(predictions_per_tree, 95, axis=0)

# Display bounds
print(lower_bound, mean_prediction, upper_bound)

[4.51482389 4.51482389 4.51482389 ... 4.83312821 4.83312821 4.83312821] [4.75842568 4.75842568 4.75842568 ... 5.11703416 5.11703416 5.11703416] [5.124825 5.124825 5.124825 ... 5.2833   5.2833   5.2833  ]


In [101]:
predictions_df['Predictions'] = mean_prediction
predictions_df['Lower Bound'] = lower_bound
predictions_df['Upper Bound'] = upper_bound

predictions_df.sample(n=5)

,Year,Height (in),Weight (lbs),Broad Jump (in),Predictions,Lower Bound,Upper Bound
62936,2024,59,291,42,5.279862,5.094396,5.426034
630241,2024,73,344,63,5.347326,5.181386,5.489463
446677,2024,69,184,19,4.764316,4.516971,5.124825
91740,2024,60,215,134,4.502751,4.422673,4.592737
544866,2024,71,307,84,5.306885,5.139555,5.424245


We now have predictions with upper and lower limits so that we an provide a range of anticipated results through PowerBI.

In [102]:
predictions_df.to_csv('resources/40_yard_predicitons.csv')